In [ ]:
#注意力机制
#seq2seq架构：编码器 解码器 中间语义张量c
#注意力机制的分类
#1、软注意力: 注意力权重值分布在0-1之间，关注所有的词汇，但是不同词汇根据权重大小关注的程度不一样。
#2、硬注意力: 注意力权重值是0或者1，只关注哪些重要的部分，忽略次要的部分
#3、自注意力: 通过输入项内部的"表决"来决定应该关注哪些输入项.
#软注意力机制 
#没有加attention前对后的影响相同
#加了attention机制后  KVQ
#硬注意力/局部注意力 部分权重为0
#自注意力机制：两两token计算注意力捕捉词语内部特征q=k=v

In [ ]:
## 四、注意力计算规则
#计算规则前提：
#必须有指定的数据: Q、K、V；当输入的Q=K=V时（或者Q\K\V来自于同一个X）, 称作自注意力计算规则；当Q、K、V不相等时称为一般注意力计算规则
#三种规则方法：
#第一种方法: 将Q和K进行纵轴拼接，然后经过线性变换，再经过Softmax进行处理得到权重，最后和V进行相乘
#第二种方法: 将Q和K进行纵轴拼接，接着经过一次线性变化，然后进过tanh激活函数处理，再进行sum求和，再经过softmax进行处理得到权重，最后
#和V进行张量的乘法
#第三种方法: 将Q和K的转置进行点乘，然后除以一个缩放系数，再经过softmax进行处理得到权重，最后和V进行张量的乘法 #除（归一化） 防止梯度消失
#同时符合正态分布
#在解码器端的注意力机制: 能够根据模型目标有效的聚焦编码器的输出结果, 当其作为解码器的输入时提升效果. 改善以往编码器输出是单一定长张量, 
#无法存储过多信息的情况.
#在编码器端的注意力机制: 主要解决表征问题, 相当于特征提取过程, 得到输入的注意力表示. 一般使用自注意力(self-attention).

In [ ]:
#注意力机制实现步骤（深度学习中）:
#第一步: 按照注意力规则，对Q、K、V进行注意力的计算
#第二步: 如果第一步是拼接操作，需要将Q和第一步计算的结果进行再次拼接，如果是点乘运算，Q和K、V相等,一般属于自注意力，不需要拼接
#第三步: 我们需要将第二步的结果，进行线性变化，按照指定输出维度进行结果的表示


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [18]:
#进行降维升维
class MyAtt1(nn.Module):
    def __init__(self,query_size,key_size,value_size1,value_size2,output_size):
        super(MyAtt,self).__init__()
        self.query_size =query_size#最后一个维度查询
        self.key_size = key_size#最后一个维度查询
        self.value_size1 = value_size1#最后中间维度查询
        self.value_size2 = value_size2#最后一个维度查询
        self.output_size = output_size#最后一个维度查询
        #QK进行拼接
        self.attn = nn.Linear(self.query_size + self.key_size,self.value_size1)
        #输出[1,1,32]-[1,1,64]
        #V[1,32,64]
        #和第一步输出的结果进行拼接[1,1,32][1,1,64]--[1,1,96]
        self.attn_combine = nn.Linear(self.query_size + self.value_size2,output_size)
    def forward(self,Q,K,V):
        #按照第一种进行计算
        #Q和K进行拼接 [1,32][1,32]→[1,64]
        attn_weights = F.softmax(self.attn(torch.cat((Q[0],K[0]),dim=-1)),dim=-1)
        #将得到的权重和V进行矩阵相乘
        #升维 [1,1,32] 最后变成[1,1,64]
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),V)
        #将Q和第一步计算的结果进行拼接
        attn_cat = torch.cat((Q[0],attn_applied[0]),dim = -1)
        #按照指定输出
        output = self.attn_combine(attn_cat).unsqueeze(0)
        return output,attn_weights
#Q = torch.randn((1,1,32))
#K = torch.randn((1,1,32))
#V = torch.randn((1,32,64))
#my_att = MyAtt1(32,32,32,64,32)
#output,attn_weights = my_att(Q,K,V)
#print(output.shape)
#print(attn_weights.shape)

torch.Size([1, 1, 32])
torch.Size([1, 32])


In [22]:
#不进行降维升维的放法
class MyAtt2(nn.Module):
    def __init__(self,query_size,key_size,value_size1,value_size2,output_size):
        super(MyAtt2,self).__init__()
        self.query_size =query_size#最后一个维度查询
        self.key_size = key_size#最后一个维度查询
        self.value_size1 = value_size1#最后中间维度查询
        self.value_size2 = value_size2#最后一个维度查询
        self.output_size = output_size#最后一个维度查询
        #QK进行拼接
        self.attn = nn.Linear(self.query_size + self.key_size,self.value_size1)
        #输出[1,1,32]-[1,1,64]
        #V[1,32,64]
        #和第一步输出的结果进行拼接[1,1,32][1,1,64]--[1,1,96]
        self.attn_combine = nn.Linear(self.query_size + self.value_size2,output_size)
    def forward(self,Q,K,V):
        #按照第一种进行计算
        #Q和K进行拼接 [1,1,32][1,1,32]→[1,1,64]
        attn_weights = F.softmax(self.attn(torch.cat((Q,K),dim=-1)),dim=-1)
        #将得到的权重和V进行矩阵相乘
        #升维 [1,1,32] 最后变成[1,1,64]
        attn_applied = torch.bmm(attn_weights,V)
        #将Q和第一步计算的结果进行拼接
        attn_cat = torch.cat((Q,attn_applied),dim = -1)
        #按照指定输出
        output = self.attn_combine(attn_cat)
        return output,attn_weights
Q = torch.randn((1,1,32))
K = torch.randn((1,1,32))
V = torch.randn((1,32,64))
my_att2 = MyAtt2(32,32,32,64,32)
output,attn_weights = my_att2(Q,K,V)
print(output)
print(attn_weights.shape)

tensor([[[-0.4907, -0.0493,  0.0256, -0.1935, -0.1729,  0.0537,  0.3131,
          -0.1877, -0.2340, -0.2544,  0.1078, -0.2232, -0.3326, -0.0251,
           0.0693, -0.2420, -0.3071, -0.5321, -0.0838, -0.1780, -0.2170,
           0.3604, -0.1808,  0.6961, -0.0216, -0.0676, -0.2175, -0.5165,
          -0.0329,  0.0484,  0.2552,  0.2398]]], grad_fn=<ViewBackward0>)
torch.Size([1, 1, 32])
